In [1]:
import keras as k
import numpy
import os

Using TensorFlow backend.


In [2]:
with open("example-keras-model-files/KERAS_conv2d_model.json") as jsonFile:
    jsonModel = jsonFile.read()
model = k.models.model_from_json(jsonModel)
model.load_weights("example-keras-model-files/KERAS_conv2d_model_weights.h5")
model.summary()

FileNotFoundError: [Errno 2] No such file or directory: 'example-keras-model-files/KERAS_conv2d.json'